# Import data to sgkit

Convert MalariaGEN data from scikit-allel VCF Zarr format, to sgkit Zarr format. This uses the `vcfzarr_to_zarr` function that has been optimized to avoid high-memory usage. (See https://github.com/pystatgen/sgkit/pull/324 and the linked issues for details.)

We also use the [rechunker](https://rechunker.readthedocs.io/en/latest/) library to rechunk to the desired chunk sizes.

In [1]:
%run setup.ipynb

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sgkit as sg
from dask.diagnostics import ProgressBar
from sgkit.io.vcfzarr_reader import vcfzarr_to_zarr

In [4]:
input = here() / 'data/external/ag1000g/phase2/AR1/variation/main/zarr/pass/ag1000g.phase2.ar1.pass/'
output = here() / 'data/sgkit/ag1000g_import.zarr'

In [5]:
with ProgressBar():
    vcfzarr_to_zarr(input, output, grouped_by_contig=True, consolidated=True)

[########################################] | 100% Completed |  5.2s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 30.9s
[########################################] | 100% Completed |  7.5s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 39.5s
[########################################] | 100% Completed |  4.9s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 29.2s
[########################################] | 100% Completed |  6.6s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 36.8s
[########################################] | 100% Completed |  2.7s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

Have a look at the dataset that's been created, by reading it with Xarray (using the sgkit convenience function).

In [6]:
ds = sg.load_dataset(output)
ds

,Array,Chunk
Bytes,132.10 GB,63.96 MB
Shape,"(57837885, 1142, 2)","(524288, 61, 2)"
Count,2110 Tasks,2109 Chunks
Type,int8,numpy.ndarray
,Array,Chunk
Bytes,132.10 GB,63.96 MB
Shape,"(57837885, 1142, 2)","(524288, 61, 2)"
Count,2110 Tasks,2109 Chunks
Type,bool,numpy.ndarray
,Array,Chunk


The dataset is chunked, in the `samples` dimension (chunk size is `61`, total number of samples is `1142`), but the popgen functions don't support chunking in that dimension, so we need to rechunk to have a single chunk in that dimension. We can do that using [rechunker](https://rechunker.readthedocs.io/en/latest/) working directly on Zarr groups.

In [7]:
source_group = zarr.open(str(output))
target_chunks = {"call_genotype": (524288, 1142, 2), "call_genotype_mask": (524288, 1142, 2), "sample_id": None, "variant_allele": None, "variant_contig": None, "variant_position": None}
max_mem = '2GB'

target_store = str(here() / 'data/sgkit/ag1000g.zarr')
temp_store = str(here() / 'data/sgkit/ag1000g_rechunked_tmp.zarr')

In [8]:
from rechunker import api as rechunker_api
plan = rechunker_api.rechunk(source_group, target_chunks, max_mem, target_store)

In [9]:
with ProgressBar():
    plan.execute()

[########################################] | 100% Completed |  3min  9.5s


Now when we look at the dataset it is chunked as we want it.

In [10]:
ds = sg.load_dataset(target_store)
ds

<xarray.Dataset>
Dimensions:             (alleles: 4, ploidy: 2, samples: 1142, variants: 57837885)
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    call_genotype       (variants, samples, ploidy) int8 dask.array<chunksize=(524288, 1142, 2), meta=np.ndarray>
    call_genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(524288, 1142, 2), meta=np.ndarray>
    sample_id           (samples) <U1 dask.array<chunksize=(1142,), meta=np.ndarray>
    variant_allele      (variants, alleles) |S1 dask.array<chunksize=(4194304, 4), meta=np.ndarray>
    variant_contig      (variants) int32 dask.array<chunksize=(4194304,), meta=np.ndarray>
    variant_position    (variants) int32 dask.array<chunksize=(4194304,), meta=np.ndarray>
Attributes:
    contigs:                    ['2L', '2R', '3L', '3R', 'X']
    max_variant_allele_length:  1